In [23]:
import pandas as pd
import imageio
import json
from datetime import datetime
import pytz


from render_video import plot_figures, figs_to_frames

## Testlauf Neupfarrplatz

In [24]:
# load parquet file to dataframe
df = pd.read_parquet('../data/testlauf-neupfarrplatz/log-interpol/interpol.parquet')

In [25]:
df.head()

team 50b848eb-150d-11ef-a477-15bc960d382e             \
dim                                   lat        lon   
0                               49.018418  12.096596   
1000                            49.018418  12.096595   
2000                            49.018418  12.096595   
3000                            49.018418  12.096595   
4000                            49.018418  12.096595   

team a4eebb09-1514-11ef-b019-73f1875d90c5             \
dim                                   lat        lon   
0                               49.018377  12.096603   
1000                            49.018377  12.096603   
2000                            49.018377  12.096603   
3000                            49.018377  12.096603   
4000                            49.018377  12.096603   

team b1d91f00-1514-11ef-90d9-870e7bed4f26             \
dim                                   lat        lon   
0                               49.018386  12.096590   
1000                            49.018388  12.096587   
2000                            49.018388  12.096585   
3000                            49.018388  12.096585   
4000                            49.018388  12.096584   

team 56971933-150d-11ef-9427-2bf640545472             \
dim                                   lat        lon   
0                               49.018445  12.096630   
1000                            49.018444  12.096632   
2000                            49.018442  12.096636   
3000                            49.018440  12.096640   
4000                            49.018438  12.096644   

team 284f0e6b-1514-11ef-a3d1-710a0d2c97dd             
dim                                   lat        lon  
0                               49.018446  12.096713  
1000                            49.018445  12.096713  
2000                            49.018445  12.096713  
3000                            49.018445  12.096713  
4000                            49.018445  12.096714

In [26]:
# get list of teams
teams = list(set(c[0] for c in df.columns))
teams

['a4eebb09-1514-11ef-b019-73f1875d90c5',
 '50b848eb-150d-11ef-a477-15bc960d382e',
 '56971933-150d-11ef-9427-2bf640545472',
 '284f0e6b-1514-11ef-a3d1-710a0d2c97dd',
 'b1d91f00-1514-11ef-90d9-870e7bed4f26']

In [27]:
# get catch data
data_caught = json.load(open('../data/testlauf-neupfarrplatz/log-export/team_caught.json'))

In [28]:
# dictionary of caught timestamps
caught_timestamps = {}
for entry in data_caught:
    entry['timestamp'] = entry['timestamp'][:-4]
    entry['timestamp'] = entry['timestamp']
    dt = datetime.strptime(entry['timestamp'], "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.UTC)
    caught_timestamps[entry['runaway_active_user']] = int(dt.timestamp() * 1000)
# initial hunting team
entry = data_caught[-1]
caught_timestamps

{'56971933-150d-11ef-9427-2bf640545472': 1716041758000,
 'b1d91f00-1514-11ef-90d9-870e7bed4f26': 1716041166000,
 'a4eebb09-1514-11ef-b019-73f1875d90c5': 1716039541000}

In [29]:
# get initial hunter
initial_hunter = entry['hunter_active_user']
caught_timestamps[initial_hunter] = 0

In [30]:
# Render the frames figures
figs = plot_figures(df, teams, initial_hunter, caught_timestamps)

100%|██████████| 893/893 [00:45<00:00, 19.43it/s]


In [31]:
# Convert figures to images
frames = figs_to_frames(figs)

100%|██████████| 893/893 [17:54<00:00,  1.20s/it]


In [32]:
# repeat last frame for 2 seconds
for i in range(60):
    frames.append(frames[-1])

In [33]:
# Save frames as a video
imageio.mimsave("video_neupfarrplatz.mp4", frames, fps=30)